In [ ]:
import os
import random
import json
from transformers import AutoTokenizer
from datasets import load_from_disk, load_dataset, DatasetDict
from Libraries import Summarizer_Trainer as Trainer

In [ ]:
# ============================================================
# 1️⃣  CẤU HÌNH BIẾN TOÀN CỤC
# ============================================================
DATABASE = "Database"
DATASET = "Datasets/vietnews"
MODELS = "Models"
Service = "Summarizer"
DataFile = "TrainData.jsonl"
ModelName = "bartpho-summarizer"
Checkpoint = "vinai/bartpho-syllable"

DataPath = f"{DATABASE}/{Service}/{DataFile}"
ModelPath = f"{MODELS}/{Service}/{ModelName}"

# ⚙️ THÔNG SỐ HUẤN LUYỆN — TỐI ƯU CHO RTX 4050 8GB
MAX_INPUT_LENGTH = 512          # rút ngắn để giảm bộ nhớ & tăng tốc
MAX_TARGET_LENGTH = 128
NUM_TRAIN_EPOCHS = 3            # khoảng 8–10h với cấu hình này
LEARNING_RATE = 3e-5
WEIGHT_DECAY = 0.01
BATCH_SIZE = 2                  # vừa khít GPU 8GB khi fp16 + grad_accum=4

In [ ]:
if not os.path.exists(DATASET):
    ds = load_dataset("nam194/vietnews")
    ds.save_to_disk("Datasets/vietnews")

In [ ]:
# ============================================================
# 2️⃣  LOAD DATASET
# ============================================================
print("🔹 Đang tải dataset ...")
if os.path.isdir(DATASET):
    dataset = load_from_disk(DATASET)
elif os.path.exists(DataPath):
    from datasets import load_dataset
    dataset = load_dataset("json", data_files=DataPath)
else:
    raise FileNotFoundError(f"❌ Không tìm thấy dataset tại {DATASET} hoặc {DataPath}")

print(f"✅ Dataset loaded thành công: {dataset}")
if isinstance(dataset, DatasetDict):
    print(f"📦 Các split: {list(dataset.keys())}")

In [ ]:
# ============================================================
# 3️⃣  TOKENIZER & TRAINER KHỞI TẠO
# ============================================================
print(f"🔹 Đang tải tokenizer từ checkpoint: {Checkpoint}")
tokenizer = AutoTokenizer.from_pretrained(Checkpoint)

summarizer_trainer = Trainer.SummarizationTrainer(
    # ===== DỮ LIỆU =====
    Max_Input_Length=MAX_INPUT_LENGTH,
    Max_Target_Length=MAX_TARGET_LENGTH,
    prefix="",                          # BARTPho không cần tiền tố
    input_column="article",
    target_column="abstract",           # VietNews dùng cột "abstract"

    # ===== HUẤN LUYỆN =====
    Learning_Rate=LEARNING_RATE,
    Weight_Decay=WEIGHT_DECAY,
    Batch_Size=BATCH_SIZE,
    Num_Train_Epochs=NUM_TRAIN_EPOCHS,
    gradient_accumulation_steps=4,      # → effective batch = 2×4 = 8
    warmup_ratio=0.1,                   # tăng dần LR trong 10% đầu
    lr_scheduler_type="linear",
    seed=42,

    # ===== SUY DIỄN / SINH =====
    num_beams=4,                        # trade-off giữa chất lượng & tốc độ
    fp16=True,                          # giảm VRAM, tăng tốc
    early_stopping_patience=1,          # dừng sớm nếu không cải thiện
    logging_steps=500,
    report_to="none",                   # tắt logging ngoài
)

In [ ]:
# ============================================================
# 4️⃣  CHẠY HUẤN LUYỆN
# ============================================================
print("\n🚀 Bắt đầu huấn luyện ...\n")
trainer = summarizer_trainer.run(
    Checkpoint=Checkpoint,
    ModelPath=ModelPath,
    DataPath=DATASET,
    tokenizer=tokenizer,
)
print("\n✅ Huấn luyện hoàn tất.")

In [ ]:
# ============================================================
# 5️⃣  KIỂM THỬ NHANH
# ============================================================
if "test" in dataset:
    sample = random.choice(dataset["test"])
elif "validation" in dataset:
    sample = random.choice(dataset["validation"])
else:
    sample = random.choice(dataset["train"])

article_text = sample["article"]
print("\n📰 Bài báo gốc:\n")
print(article_text[:700], "...")

summary = summarizer_trainer.generate(article_text, max_new_tokens=160)
print("\n🧾 Tóm tắt mô hình sinh ra:\n")
print(summary)

In [ ]:
# ============================================================
# 6️⃣  ĐÁNH GIÁ & LƯU KẾT QUẢ
# ============================================================
print("\n📊 Đang đánh giá mô hình ...")
eval_results = trainer.evaluate()
results_path = os.path.join(ModelPath, "eval_results.json")

with open(results_path, "w", encoding="utf-8") as f:
    json.dump(eval_results, f, ensure_ascii=False, indent=2)

print(f"💾 Kết quả ROUGE đã lưu tại: {results_path}")
print("\n🎯 Pipeline hoàn tất.")